In [3]:
import pandas as pd
import re

# Путь к файлу
file_path = 'data_analysis/S11_evening Tram vehicles.att'

# Поиск строки заголовка
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Считываем файл построчно
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Определяем индекс строки заголовка
for i, line in enumerate(lines):
    if header_line in line:
        header_index = i
        break
else:
    raise ValueError("Заголовок не найден в файле.")

# Загружаем данные в DataFrame, начиная со строки заголовка
from io import StringIO
data_str = ''.join(lines[header_index:])
df = pd.read_csv(StringIO(data_str), sep=';')

# Функция для извлечения минимальных подстрок перед символами "@" после "-"
def extract_links(movement_str):
    try:
        # Найти все вхождения вида "число@..." после "-"
        matches = re.findall(r'-(\s*\d+)@', movement_str)
        if len(matches) < 2:
            return pd.Series([None, None])
        link_from = int(matches[0].strip())
        link_to = int(matches[1].strip())
        return pd.Series([link_from, link_to])
    except Exception:
        return pd.Series([None, None])

# Применяем функцию к столбцу MOVEMENT
df[['LinkFrom', 'LinkTo']] = df['MOVEMENT'].apply(extract_links)

# Сохраняем результат в Excel
output_df = df[['MOVEMENT', 'LinkFrom', 'LinkTo']]
output_df.to_excel('movement_links.xlsx', index=False)

print("Файл movement_links.xlsx успешно создан.")


Файл movement_links.xlsx успешно создан.


In [4]:
import pandas as pd
import re
from io import StringIO

# Путь к файлу
file_path = "data_analysis/S11_evening Tram vehicles.att"  # Замените на нужный путь, если отличается

# Строка заголовка
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Считываем файл построчно
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Поиск строки заголовка
for i, line in enumerate(lines):
    if header_line in line:
        header_index = i
        break
else:
    raise ValueError("Заголовок не найден в файле.")

# Загрузка данных начиная с заголовка
data_str = ''.join(lines[header_index:])
df = pd.read_csv(StringIO(data_str), sep=';')

# Преобразование SIMRUN в числовой тип
simrun_column = "$MOVEMENTEVALUATION:SIMRUN"
df[simrun_column] = pd.to_numeric(df[simrun_column], errors='coerce')

# Фильтрация по максимальному SIMRUN
max_simrun = df[simrun_column].max()
df_filtered = df[df[simrun_column] == max_simrun].copy()

# Функция для извлечения минимальных подстрок перед '@' после '-'
def extract_links(movement_str):
    try:
        matches = re.findall(r'-(\\s*\\d+)@', movement_str)
        if len(matches) < 2:
            return pd.Series([None, None])
        link_from = int(matches[0].strip())
        link_to = int(matches[1].strip())
        return pd.Series([link_from, link_to])
    except Exception:
        return pd.Series([None, None])

# Применяем функцию к MOVEMENT
df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].apply(lambda x: extract_links(str(x)))

# Отбираем нужные столбцы
columns_needed = [
    'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

output_df = df_filtered[columns_needed]

# Сохраняем результат
output_df.to_excel('movement_links_filtered.xlsx', index=False)

print("Файл 'movement_links_filtered.xlsx' успешно создан.")


Файл 'movement_links_filtered.xlsx' успешно создан.


In [6]:
import pandas as pd
import re
from io import StringIO

# Путь к файлу .att
file_path = "data_analysis/S11_evening Tram vehicles.att"  # Укажите верный путь к файлу

# Строка, обозначающая начало таблицы
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Считываем файл построчно
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Определение строки заголовка
for i, line in enumerate(lines):
    if header_line in line:
        header_index = i
        break
else:
    raise ValueError("Строка заголовка не найдена.")

# Загрузка таблицы с заголовком
data_str = ''.join(lines[header_index:])
df = pd.read_csv(StringIO(data_str), sep=';')

# Преобразование $MOVEMENTEVALUATION:SIMRUN в числовой тип и фильтрация по максимуму
simrun_column = "$MOVEMENTEVALUATION:SIMRUN"
df[simrun_column] = pd.to_numeric(df[simrun_column], errors='coerce')
max_simrun = df[simrun_column].max()
df_filtered = df[df[simrun_column] == max_simrun].copy()

# Извлечение LinkFrom и LinkTo: минимальные подстроки перед первым и вторым @, ограниченные слева символом "-"
def extract_links(movement_str):
    try:
        # Ищем шаблоны вида "-<число>@"
        matches = re.findall(r'-(\\d+)@', movement_str)
        if len(matches) < 2:
            return pd.Series([None, None])
        return pd.Series([int(matches[0]), int(matches[1])])
    except Exception:
        return pd.Series([None, None])

# Применяем функцию
df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

# Столбцы, которые нужно оставить
columns_needed = [
    'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Оставляем только нужные столбцы
output_df = df_filtered[columns_needed].copy()

# Заменяем запятые на точки в числовых столбцах
for col in columns_needed[3:]:
    output_df[col] = output_df[col].astype(str).str.replace(',', '.', regex=False)

# Сохраняем результат
output_df.to_excel("movement_links_filtered.xlsx", index=False)

print("Файл 'movement_links_filtered.xlsx' успешно создан.")


Файл 'movement_links_filtered.xlsx' успешно создан.


In [7]:
import pandas as pd
import re
from io import StringIO

# Путь к файлу .att
file_path = "data_analysis/S11_evening Tram vehicles.att"  # Замените на актуальный путь

# Строка, обозначающая начало таблицы
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Считываем файл построчно
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Поиск строки заголовка
for i, line in enumerate(lines):
    if header_line in line:
        header_index = i
        break
else:
    raise ValueError("Строка заголовка не найдена.")

# Загрузка таблицы
data_str = ''.join(lines[header_index:])
df = pd.read_csv(StringIO(data_str), sep=';')

# Преобразуем $MOVEMENTEVALUATION:SIMRUN в числовой тип
simrun_column = "$MOVEMENTEVALUATION:SIMRUN"
df[simrun_column] = pd.to_numeric(df[simrun_column], errors='coerce')
max_simrun = df[simrun_column].max()
df_filtered = df[df[simrun_column] == max_simrun].copy()

# Функция извлечения LinkFrom и LinkTo
def extract_links(movement_str):
    try:
        matches = re.findall(r'-(\d+)@', movement_str)
        if len(matches) >= 2:
            return pd.Series([int(matches[0]), int(matches[1])])
    except Exception:
        pass
    return pd.Series([None, None])

# Применяем функцию
df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

# Столбцы для вывода
columns_needed = [
    'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Оставляем нужные столбцы
output_df = df_filtered[columns_needed].copy()

# Заменяем запятые на точки в числовых значениях
for col in columns_needed[3:]:
    output_df[col] = output_df[col].astype(str).str.replace(",", ".", regex=False)

# Сохраняем результат
output_df.to_excel("movement_links_filtered.xlsx", index=False)

print("Файл 'movement_links_filtered.xlsx' успешно создан.")


Файл 'movement_links_filtered.xlsx' успешно создан.


In [8]:
import pandas as pd
import re
from io import StringIO
import os

# Список файлов .att
file_list = [
    "data_analysis/S11_evening Tram vehicles.att",
    "data_analysis/S21_evening NewTram vehicles.att"
]

# Строка, указывающая начало таблицы
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Столбцы, которые нужно сохранить
columns_needed = [
    'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Результаты для записи в Excel
output_dfs = {}

# Обработка каждого файла
for file_path in file_list:
    file_name = os.path.basename(file_path).replace('.att', '')  # Имя файла без расширения

    # Считываем файл построчно
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Поиск строки заголовка
    for i, line in enumerate(lines):
        if header_line in line:
            header_index = i
            break
    else:
        raise ValueError(f"Заголовок не найден в файле: {file_path}")

    # Загрузка данных в DataFrame
    data_str = ''.join(lines[header_index:])
    df = pd.read_csv(StringIO(data_str), sep=';')

    # Преобразование SIMRUN в числовой тип
    simrun_column = "$MOVEMENTEVALUATION:SIMRUN"
    df[simrun_column] = pd.to_numeric(df[simrun_column], errors='coerce')
    max_simrun = df[simrun_column].max()
    df_filtered = df[df[simrun_column] == max_simrun].copy()

    # Извлечение LinkFrom и LinkTo из MOVEMENT
    def extract_links(movement_str):
        try:
            matches = re.findall(r'-(\d+)@', movement_str)
            if len(matches) >= 2:
                return pd.Series([int(matches[0]), int(matches[1])])
        except Exception:
            pass
        return pd.Series([None, None])

    df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

    # Оставляем только нужные столбцы
    output_df = df_filtered[columns_needed].copy()

    # Замена запятых на точки
    for col in columns_needed[3:]:
        output_df[col] = output_df[col].astype(str).str.replace(",", ".", regex=False)

    # Сохраняем для записи в Excel
    output_dfs[file_name] = output_df

# Запись в Excel с двумя вкладками
with pd.ExcelWriter("movement_links_combined.xlsx") as writer:
    for sheet_name, df in output_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Файл 'movement_links_combined.xlsx' успешно создан с двумя вкладками.")


Файл 'movement_links_combined.xlsx' успешно создан с двумя вкладками.


In [14]:
import pandas as pd
import re
from io import StringIO
import os

# Файлы .att
att_files = [
    "data_analysis/S11_evening Tram vehicles.att",
    "data_analysis/S21_evening NewTram vehicles.att"
]

# Загрузка списка пар для фильтрации
comparison_df = pd.read_excel("ComparisonList.xlsx", sheet_name="ATT_directions")
comparison_df['LinkFrom'] = comparison_df['LinkFrom'].astype(int)
comparison_df['LinkTo'] = comparison_df['LinkTo'].astype(int)

# Строка заголовка
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Столбцы, которые нужно вывести
columns_needed = [
    'Name', 'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Словарь с результатами
output_dfs = {}

# Обработка каждого .att файла
for file_path in att_files:
    file_name = os.path.basename(file_path).replace('.att', '')

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if header_line in line:
            header_index = i
            break
    else:
        raise ValueError(f"Заголовок не найден в {file_path}")

    data_str = ''.join(lines[header_index:])
    df = pd.read_csv(StringIO(data_str), sep=';')

    # Преобразование SIMRUN и фильтрация
    simrun_column = "$MOVEMENTEVALUATION:SIMRUN"
    df[simrun_column] = pd.to_numeric(df[simrun_column], errors='coerce')
    max_simrun = df[simrun_column].max()
    df_filtered = df[df[simrun_column] == max_simrun].copy()

    # Извлечение LinkFrom и LinkTo
    def extract_links(movement_str):
        try:
            matches = re.findall(r'-(\d+)@', movement_str)
            if len(matches) >= 2:
                return pd.Series([int(matches[0]), int(matches[1])])
        except Exception:
            pass
        return pd.Series([None, None])

    df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

    # Соединяем с таблицей фильтра
    merged = df_filtered.merge(comparison_df, on=['LinkFrom', 'LinkTo'], how='inner')

    # Заменяем запятые на точки в числовых столбцах
    for col in [
        'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
        'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
    ]:
        merged[col] = merged[col].astype(str).str.replace(",", ".", regex=False)

    # Оставляем нужные столбцы и сохраняем
    output_dfs[file_name] = merged[columns_needed]

# Запись в Excel
with pd.ExcelWriter("movement_links_filtered.xlsx") as writer:
    for sheet_name, df in output_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Файл 'movement_links_filtered.xlsx' успешно создан.")


Файл 'movement_links_filtered.xlsx' успешно создан.


In [2]:
import pandas as pd
import re
from io import StringIO
import os

# Файл со списком пар
comparison_list_path = "ComparisonList.xlsx"

# Загрузка таблицы анализа и направлений
analysis_df = pd.read_excel(comparison_list_path, sheet_name="ATT_analysis")
directions_df = pd.read_excel(comparison_list_path, sheet_name="ATT_directions")
directions_df['LinkFrom'] = directions_df['LinkFrom'].astype(int)
directions_df['LinkTo'] = directions_df['LinkTo'].astype(int)

# Строка заголовка в .att
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Поля для вывода
columns_needed = [
    'Name', 'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Обработка одного .att файла
def process_att_file(file_path, directions_df):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if header_line in line:
            header_index = i
            break
    else:
        raise ValueError(f"Заголовок не найден в {file_path}")

    data_str = ''.join(lines[header_index:])
    df = pd.read_csv(StringIO(data_str), sep=';')

    # Фильтрация по максимальному SIMRUN
    simrun_col = "$MOVEMENTEVALUATION:SIMRUN"
    df[simrun_col] = pd.to_numeric(df[simrun_col], errors='coerce')
    max_simrun = df[simrun_col].max()
    df_filtered = df[df[simrun_col] == max_simrun].copy()

    # Извлечение LinkFrom и LinkTo
    def extract_links(movement_str):
        try:
            matches = re.findall(r'-(\d+)@', movement_str)
            if len(matches) >= 2:
                return pd.Series([int(matches[0]), int(matches[1])])
        except Exception:
            pass
        return pd.Series([None, None])

    df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

    # Объединяем с направлениями
    merged = df_filtered.merge(directions_df, on=['LinkFrom', 'LinkTo'], how='inner')

    # Заменяем запятые на точки
    for col in columns_needed[4:]:
        merged[col] = merged[col].astype(str).str.replace(",", ".", regex=False)

    return merged[columns_needed]

# Обработка всех пар
for _, row in analysis_df.iterrows():
    base_file_path = os.path.join("data_analysis", row['Base'])
    comp_file_path = os.path.join("data_analysis", row['Comparison'])
    output_filename = f"{row['Filename']}.xlsx"

    output_dfs = {}

    try:
        base_name = row['Base'].replace('.att', '')
        comp_name = row['Comparison'].replace('.att', '')

        output_dfs[base_name[:31]] = process_att_file(base_file_path, directions_df)
        output_dfs[comp_name[:31]] = process_att_file(comp_file_path, directions_df)
    except Exception as e:
        print(f"⚠️ Ошибка при обработке пары: {row['Base']} / {row['Comparison']}: {e}")
        continue

    # Запись в Excel с ограничением длины имени листа
    with pd.ExcelWriter(output_filename) as writer:
        for sheet_name, df in output_dfs.items():
            safe_name = sheet_name[:31]  # Excel ограничение
            df.to_excel(writer, sheet_name=safe_name, index=False)

    print(f"✅ Файл '{output_filename}' успешно создан.")


✅ Файл 'ATT_S11_S12_evening_AddStopsTram_evening.xlsx' успешно создан.
✅ Файл 'ATT_S11_S12_morning_AddStopsTram_morning.xlsx' успешно создан.
✅ Файл 'ATT_S11_S21_evening_NewVsCurrentTramTime_evening.xlsx' успешно создан.
✅ Файл 'ATT_S11_S21_morning_NewVsCurrentTramTime_morning.xlsx' успешно создан.
✅ Файл 'ATT_S21_S22_evening_NewTramStopsTime_evening.xlsx' успешно создан.
✅ Файл 'ATT_S21_S22_morning_NewTramStopsTime_morning.xlsx' успешно создан.
✅ Файл 'ATT_S11_S30_evening_BusAreaDelayTime_evening.xlsx' успешно создан.
✅ Файл 'ATT_S11_S30_morning_BusAreaDelayTIme_morning.xlsx' успешно создан.
✅ Файл 'ATT_S11_S40_evening_PedCrossVehicleTime_evening.xlsx' успешно создан.
✅ Файл 'ATT_S11_S40_morning_PedCrossVehicleTime_morning.xlsx' успешно создан.


In [3]:
import pandas as pd
import re
from io import StringIO
import os

# Путь к файлу со списками пар
comparison_list_path = "ComparisonList.xlsx"

# Загрузка таблиц
analysis_df = pd.read_excel(comparison_list_path, sheet_name="ATT_analysis")
directions_df = pd.read_excel(comparison_list_path, sheet_name="ATT_directions")
directions_df['LinkFrom'] = directions_df['LinkFrom'].astype(int)
directions_df['LinkTo'] = directions_df['LinkTo'].astype(int)

# Строка заголовка
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Поля для экспорта
columns_needed = [
    'Name', 'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Папка для выходных файлов
output_dir = "ATT_analysis"
os.makedirs(output_dir, exist_ok=True)

# Обработка одного .att файла
def process_att_file(file_path, directions_df):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if header_line in line:
            header_index = i
            break
    else:
        raise ValueError(f"Заголовок не найден в {file_path}")

    data_str = ''.join(lines[header_index:])
    df = pd.read_csv(StringIO(data_str), sep=';')

    # Преобразование SIMRUN и фильтрация
    simrun_col = "$MOVEMENTEVALUATION:SIMRUN"
    df[simrun_col] = pd.to_numeric(df[simrun_col], errors='coerce')
    max_simrun = df[simrun_col].max()
    df_filtered = df[df[simrun_col] == max_simrun].copy()

    # Извлечение LinkFrom и LinkTo
    def extract_links(movement_str):
        try:
            matches = re.findall(r'-(\d+)@', movement_str)
            if len(matches) >= 2:
                return pd.Series([int(matches[0]), int(matches[1])])
        except Exception:
            pass
        return pd.Series([None, None])

    df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

    # Объединение с фильтрующим списком направлений
    merged = df_filtered.merge(directions_df, on=['LinkFrom', 'LinkTo'], how='inner')

    # Замена запятых на точки
    for col in columns_needed[4:]:
        merged[col] = merged[col].astype(str).str.replace(",", ".", regex=False)

    return merged[columns_needed]

# Основной цикл по всем парам
for _, row in analysis_df.iterrows():
    base_file_path = os.path.join("data_analysis", row['Base'])
    comp_file_path = os.path.join("data_analysis", row['Comparison'])
    output_filename = os.path.join(output_dir, f"{row['Filename']}.xlsx")

    output_dfs = {}

    try:
        base_name = row['Base'].replace('.att', '')[:31]
        comp_name = row['Comparison'].replace('.att', '')[:31]

        output_dfs[base_name] = process_att_file(base_file_path, directions_df)
        output_dfs[comp_name] = process_att_file(comp_file_path, directions_df)
    except Exception as e:
        print(f"⚠️ Ошибка при обработке пары: {row['Base']} / {row['Comparison']}: {e}")
        continue

    # Сохраняем результат
    with pd.ExcelWriter(output_filename) as writer:
        for sheet_name, df in output_dfs.items():
            writer.write_cells = False
            safe_sheet = sheet_name[:31]  # Excel ограничение
            df.to_excel(writer, sheet_name=safe_sheet, index=False)

    print(f"✅ Файл сохранён: {output_filename}")


✅ Файл сохранён: ATT_analysis\ATT_S11_S12_evening_AddStopsTram.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S12_morning_AddStopsTram.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S21_evening_NewVsCurrentTramTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S21_morning_NewVsCurrentTramTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S21_S22_evening_NewTramStopsTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S21_S22_morning_NewTramStopsTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S30_evening_BusAreaDelayTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S30_morning_BusAreaDelayTIme.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S40_evening_PedCrossVehicleTime.xlsx
✅ Файл сохранён: ATT_analysis\ATT_S11_S40_morning_PedCrossVehicleTime.xlsx


In [1]:
import pandas as pd
import re
from io import StringIO
import os

# Path to the Excel file with the list of file pairs
comparison_list_path = "ComparisonList.xlsx"

# Load file pair definitions and link direction filters
analysis_df = pd.read_excel(comparison_list_path, sheet_name="ATT_analysis")
directions_df = pd.read_excel(comparison_list_path, sheet_name="ATT_directions")
directions_df['LinkFrom'] = directions_df['LinkFrom'].astype(int)
directions_df['LinkTo'] = directions_df['LinkTo'].astype(int)

# Header line that indicates the start of the table in .att files
header_line = "$MOVEMENTEVALUATION:SIMRUN;TIMEINT;MOVEMENT;QLEN;QLENMAX;VEHS(ALL);VEHS(50);PERS(ALL);PERS(50);LOS(ALL);LOS(50);LOSVAL(ALL);LOSVAL(50);VEHDELAY(ALL);VEHDELAY(50);PERSDELAY(ALL);PERSDELAY(50);STOPDELAY(ALL);STOPDELAY(50);STOPS(ALL);STOPS(50);EMISSIONSCO;EMISSIONSNOX;EMISSIONSVOC;FUELCONSUMPTION"

# Columns to include in the output
columns_needed = [
    'Name', 'MOVEMENT', 'LinkFrom', 'LinkTo',
    'QLEN', 'QLENMAX', 'VEHS(ALL)', 'PERS(ALL)', 'LOSVAL(ALL)',
    'VEHDELAY(ALL)', 'PERSDELAY(ALL)', 'STOPDELAY(ALL)', 'STOPS(ALL)'
]

# Output directory
output_dir = "ATT_analysis"
os.makedirs(output_dir, exist_ok=True)

# Function to process a single .att file
def process_att_file(file_path, directions_df):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if header_line in line:
            header_index = i
            break
    else:
        raise ValueError(f"Header not found in {file_path}")

    data_str = ''.join(lines[header_index:])
    df = pd.read_csv(StringIO(data_str), sep=';')

    # Filter only rows with max SIMRUN
    simrun_col = "$MOVEMENTEVALUATION:SIMRUN"
    df[simrun_col] = pd.to_numeric(df[simrun_col], errors='coerce')
    max_simrun = df[simrun_col].max()
    df_filtered = df[df[simrun_col] == max_simrun].copy()

    # Extract LinkFrom and LinkTo from MOVEMENT column
    def extract_links(movement_str):
        try:
            matches = re.findall(r'-(\d+)@', movement_str)
            if len(matches) >= 2:
                return pd.Series([int(matches[0]), int(matches[1])])
        except Exception:
            pass
        return pd.Series([None, None])

    df_filtered[['LinkFrom', 'LinkTo']] = df_filtered['MOVEMENT'].astype(str).apply(extract_links)

    # Merge with allowed directions
    merged = df_filtered.merge(directions_df, on=['LinkFrom', 'LinkTo'], how='inner')

    # Replace comma with dot in decimal values
    for col in columns_needed[4:]:
        merged[col] = merged[col].astype(str).str.replace(",", ".", regex=False)

    return merged[columns_needed]

# Process each file pair
for _, row in analysis_df.iterrows():
    base_file_path = os.path.join("data_analysis", row['Base'])
    comp_file_path = os.path.join("data_analysis", row['Comparison'])
    output_filename = os.path.join(output_dir, f"{row['Filename']}.xlsx")

    output_dfs = {}

    try:
        base_name = row['Base'].replace('.att', '')[:31]
        comp_name = row['Comparison'].replace('.att', '')[:31]

        output_dfs[base_name] = process_att_file(base_file_path, directions_df)
        output_dfs[comp_name] = process_att_file(comp_file_path, directions_df)
    except Exception as e:
        print(f"Error processing pair: {row['Base']} / {row['Comparison']}: {e}")
        continue

    # Save results to Excel with max 31 characters per sheet name
    with pd.ExcelWriter(output_filename) as writer:
        for sheet_name, df in output_dfs.items():
            safe_sheet = sheet_name[:31]
            df.to_excel(writer, sheet_name=safe_sheet, index=False)

    print(f"Output file created: {output_filename}")


Output file created: ATT_analysis\ATT_S11_S12_evening_AddStopsTram.xlsx
Output file created: ATT_analysis\ATT_S11_S12_morning_AddStopsTram.xlsx
Output file created: ATT_analysis\ATT_S11_S21_evening_NewVsCurrentTramTime.xlsx
Output file created: ATT_analysis\ATT_S11_S21_morning_NewVsCurrentTramTime.xlsx
Output file created: ATT_analysis\ATT_S21_S22_evening_NewTramStopsTime.xlsx
Output file created: ATT_analysis\ATT_S21_S22_morning_NewTramStopsTime.xlsx
Output file created: ATT_analysis\ATT_S11_S30_evening_BusAreaDelayTime.xlsx
Output file created: ATT_analysis\ATT_S11_S30_morning_BusAreaDelayTIme.xlsx
Output file created: ATT_analysis\ATT_S11_S40_evening_PedCrossVehicleTime.xlsx
Output file created: ATT_analysis\ATT_S11_S40_morning_PedCrossVehicleTime.xlsx
